In [3]:
#Código criado em 7 de Outubro de 2020
#Por: github.com/ArthurHVS

In [4]:
#Bloco de imports
import numpy as np 
import matplotlib.pyplot as plt 
import json
import pandas as pd 
from alpha_vantage.timeseries import TimeSeries 
from sklearn.preprocessing import MinMaxScaler

from keras import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [6]:
#Popula os vetores, os inverte e cria um dataset com os valores.
close_array = []
date_array = []

print("Com qual ativo iremos treinar hoje?")
symb = input()
chave = '18ETRMNXVZ4FU6SQ'
ts = TimeSeries(key=chave,output_format='pandas', indexing_type='date')
myFrame, metadata = ts.get_daily_adjusted(symbol=symb.lower() + '.sao', outputsize='full')

Com qual ativo iremos treinar hoje?
            1. open  2. high  ...  7. dividend amount  8. split coefficient
date                          ...                                          
2020-10-06    24.76    26.64  ...                 0.0                   1.0
2020-10-05    24.60    24.90  ...                 0.0                   1.0
2020-10-02    25.40    25.56  ...                 0.0                   1.0
2020-10-01    25.32    26.08  ...                 0.0                   1.0
2020-09-30    24.01    25.04  ...                 0.0                   1.0
...             ...      ...  ...                 ...                   ...
2017-04-18    22.82    24.79  ...                 0.0                   1.0
2017-04-17    23.00    23.00  ...                 0.0                   1.0
2017-04-13    22.97    23.01  ...                 0.0                   1.0
2017-04-12    22.45    22.99  ...                 0.0                   1.0
2017-04-11    21.81    22.98  ...                 0.

In [ ]:
train_set = myFrame['4. close'][:400].values
test_set = myFrame[400:].values
    
#Define o formato dos vetores de treino e teste, escalonando os para valores entre 0 e 1
sc = MinMaxScaler()
train_set = train_set.reshape(-1,1)
test_set = test_set.reshape(-1,1)
train_set_scaled = sc.fit_transform(train_set)

In [4]:
#Popula os vetores x e y dos treinos, com os valores escalonados dos sets de treino e teste
x_train = []
y_train = []
for i in range(60, 400):
    x_train.append(train_set_scaled[i-60:i,0])
    y_train.append(train_set_scaled[i,0])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [5]:
#Layers da rede neural. 50 neurônios. Dropout 20% (impede overfitting).
reg = Sequential()

reg.add(LSTM(units = 50,return_sequences=True,input_shape=(x_train.shape[1],1)))
reg.add(Dropout(0.2))

reg.add(LSTM(units = 50,return_sequences=True))
reg.add(Dropout(0.2))

reg.add(LSTM(units = 50,return_sequences=True))
reg.add(Dropout(0.2))

reg.add(LSTM(units=50))
reg.add(Dropout(0.2))

#Layer output, compilação e fit do modelo. Por fim, salva o arquivo.
reg.add(Dense(units=1))
reg.compile(optimizer = 'adam',loss='mean_squared_error')
reg.fit(x_train,y_train, epochs=20, batch_size =1,verbose=2)
reg.save('savedModels/'+symb)

In [1]:
input = myFrame['Close'][len(myFrame)-len(test_set)-60:].values
input = input.reshape(-1, 1)
input = sc.transform(input)


In [ ]:
x_test = []
for i in range(60, 400):
    x_test.append(input[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
pred = reg.predict(x_test)
pred = sc.inverse_transform(pred)

In [ ]:
plt.plot(pred, color='green')
plt.plot(test_set, color='red')
plt.title('Será que foi?')
plt.show()